# **<font color="red">Agents Workflow</font>**
- Specialized agents that control the execution flow of its sub-agents.
1. `Sequential Agent`
2. `Loop Agent`
3. `Parallel Agent`


## **<font color="blue">Sequential Agents</font>**
- Then `SequentialAgent` is a workflow agent that executes its sub-agents in the order they are specified in the list. Use the `SequentialAgent` when you want the execution to occur in a fixed, strict order.
- As with other workflow agents, the `SequentialAgent` is not powered by an LLM, and is thus deterministic in how it executes. That being said, workflow agents are concerned only with their execution (i.e. in sequence), and not their internal logic; the tools or sub-agents of a workflow agent may or may not utilize LLMs.

In [6]:
import os
from config import config
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.sequential_agent import SequentialAgent

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = "gemini-2.5-flash"
APP_NAME = "sequential_agent"
USER_ID = "user_1"
SESSION_ID = "session_001"

# --- 1. Define Sub-Agents for Each Pipeline Stage ---

# Code Writer Agent
# Takes the initial specification (from user query) and writes code.
code_writer_agent = LlmAgent(
    name="CodeWriterAgent",
    model=MODEL,
    instruction="""
    You are a Python Code Generator.
    Based *only* on the user's request, write Python code that fulfills the requirement.
    Output *only* the complete Python code block, enclosed in triple backticks (```python ... ```).
    Do not add any other text before or after the code block.
    """,
    description="Writes initial Python code based on a specification.",
    output_key="generated_code"
)

# Code Reviewer Agent
# Takes the code generated by the previous agent (read from state) and provides feedback.
code_reviewer_agent = LlmAgent(
    name="CodeReviewerAgent",
    model=MODEL,
    instruction="""
    You are an expert Python Code Reviewer.
    Your task is to provide constructive feedback on the provided code.

    **Code to Review:**
    ```python
    {generated_code}
    ```

    **Review Criteria:**
    1.  **Correctness:** Does the code work as intended? Are there logic errors?
    2.  **Readability:** Is the code clear and easy to understand? Follows PEP 8 style guidelines?
    3.  **Efficiency:** Is the code reasonably efficient? Any obvious performance bottlenecks?
    4.  **Edge Cases:** Does the code handle potential edge cases or invalid inputs gracefully?
    5.  **Best Practices:** Does the code follow common Python best practices?

    **Output:**
    Provide your feedback as a concise, bulleted list. Focus on the most important points for improvement.
    If the code is excellent and requires no changes, simply state: "No major issues found."
    Output *only* the review comments or the "No major issues" statement.
    """,
    description="Reviews code and provides feedback.",
    output_key="review_comments"
)


# Code Refactorer Agent
# Takes the original code and the review comments (read from state) and refactors the code.
code_refactorer_agent = LlmAgent(
    name="CodeRefactorerAgent",
    model=MODEL,
    instruction="""
    You are a Python Code Refactoring AI.
    Your goal is to improve the given Python code based on the provided review comments.

    **Original Code:**
    ```python
    {generated_code}
    ```

    **Review Comments:**
    {review_comments}

    **Task:**
    Carefully apply the suggestions from the review comments to refactor the original code.
    If the review comments state "No major issues found," return the original code unchanged.
    Ensure the final code is complete, functional, and includes necessary imports and docstrings.

    **Output:**
    Output *only* the final, refactored Python code block, enclosed in triple backticks (```python ... ```).
    Do not add any other text before or after the code block.
    """,
    description="Refactors code based on review comments.",
    output_key="refactored_code"
)


# --- 2. Create the SequentialAgent ---
# This agent orchestrates the pipeline by running the sub_agents in order.
code_pipeline_agent = SequentialAgent(
    name="CodePipelineAgent",
    sub_agents=[code_writer_agent, code_reviewer_agent, code_refactorer_agent],
    description="Executes a sequence of code writing, reviewing, and refactoring.",
)

root_agent = code_pipeline_agent


In [7]:
import os
import asyncio

# --- Google ADK Imports ---
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

# --- Configuration ---
try:
    from config import config
    os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY
except ImportError:
    if not os.getenv("GOOGLE_API_KEY"):
        raise EnvironmentError(
            "GOOGLE_API_KEY is not set. "
            "Either provide a config.py with config.GOOGLE_API_KEY "
            "or set the environment variable directly."
        )

# --- Constants ---
MODEL = "gemini-2.5-flash"
APP_NAME = "research_report_pipeline"
USER_ID = "user_1"
SESSION_ID = "session_001"


# ===========================================================================
# PIPELINE OVERVIEW
# ===========================================================================
#
#   User Topic
#       │
#       ▼
#  ┌─────────────────────┐
#  │  ResearcherAgent    │  ── Gathers key facts, stats & perspectives
#  │  output_key:        │     on the given topic
#  │  "raw_research"     │
#  └────────┬────────────┘
#           │  raw_research → injected into next agent's prompt
#           ▼
#  ┌─────────────────────┐
#  │  AnalystAgent       │  ── Identifies trends, insights & gaps
#  │  output_key:        │     from the raw research
#  │  "analysis"         │
#  └────────┬────────────┘
#           │  raw_research + analysis → injected into next agent's prompt
#           ▼
#  ┌─────────────────────┐
#  │  ReportWriterAgent  │  ── Produces a clean, structured markdown report
#  │  output_key:        │
#  │  "final_report"     │
#  └─────────────────────┘
#
# ===========================================================================


# ---------------------------------------------------------------------------
# Stage 1 — Researcher Agent
# ---------------------------------------------------------------------------
# Receives: user's topic (from the initial query)
# Produces: raw_research  →  saved to session state

researcher_agent = LlmAgent(
    name="ResearcherAgent",
    model=MODEL,
    instruction="""You are a thorough Research Specialist.

Your task is to gather comprehensive information on the topic provided by the user.

**What to include:**
- A brief overview / background of the topic
- 5–7 key facts or statistics (with context)
- 2–3 different perspectives or schools of thought
- Notable recent developments (last 2–3 years if applicable)
- Key challenges or open questions in this area

**Output format:**
Use clear section headers and bullet points.
Output *only* the structured research notes — no preamble, no conclusion.
""",
    description="Gathers structured research notes on a given topic.",
    output_key="raw_research",
)


# ---------------------------------------------------------------------------
# Stage 2 — Analyst Agent
# ---------------------------------------------------------------------------
# Receives: raw_research  (injected from session state via {raw_research})
# Produces: analysis      →  saved to session state

analyst_agent = LlmAgent(
    name="AnalystAgent",
    model=MODEL,
    instruction="""You are a senior Data & Insights Analyst.

You have been given raw research notes. Your job is to analyse them critically.

**Raw Research Notes:**
{raw_research}

**Your analysis must cover:**
1. **Key Trends** – What patterns or directions are emerging?
2. **Strengths / Opportunities** – What positive signals exist?
3. **Weaknesses / Risks** – What concerns or limitations stand out?
4. **Knowledge Gaps** – What important questions remain unanswered?
5. **Actionable Insights** – What are the 3 most important takeaways for a decision-maker?

**Output format:**
Use numbered sections matching the five areas above.
Be concise and analytical — avoid repeating the raw research verbatim.
Output *only* the analysis. No preamble or closing remarks.
""",
    description="Analyses raw research to extract trends, risks, and insights.",
    output_key="analysis",
)


# ---------------------------------------------------------------------------
# Stage 3 — Report Writer Agent
# ---------------------------------------------------------------------------
# Receives: raw_research + analysis  (both injected from session state)
# Produces: final_report  →  saved to session state

report_writer_agent = LlmAgent(
    name="ReportWriterAgent",
    model=MODEL,
    instruction="""You are a professional Technical Report Writer.

Using the research notes and analysis provided below, write a polished, 
well-structured research report in Markdown format.

**Research Notes:**
{raw_research}

**Analysis:**
{analysis}

**Report Structure (strictly follow this):**

# [Descriptive Report Title]

## Executive Summary
A 3–4 sentence overview of the entire report.

## Background
Context and importance of the topic (2–3 short paragraphs).

## Key Findings
Present the most important facts and data points clearly.

## Analysis & Insights
Summarise the trends, opportunities, risks, and knowledge gaps.

## Recommendations
3–5 concrete, actionable recommendations based on the findings.

## Conclusion
A brief, forward-looking closing paragraph.

---
*Report generated by the Research Report Pipeline.*

**Rules:**
- Use proper Markdown (headers, bold, bullet points where appropriate).
- Do not invent facts not present in the research notes or analysis.
- Keep the tone professional and objective.
- Output *only* the Markdown report — no extra commentary.
""",
    description="Writes a polished Markdown report from research notes and analysis.",
    output_key="final_report",
)


# ---------------------------------------------------------------------------
# SequentialAgent — ties all three stages together
# ---------------------------------------------------------------------------

research_pipeline_agent = SequentialAgent(
    name="ResearchReportPipeline",
    sub_agents=[researcher_agent, analyst_agent, report_writer_agent],
    description=(
        "A three-stage pipeline that: "
        "(1) researches a topic, "
        "(2) analyses the findings, "
        "(3) writes a formatted report."
    ),
)

# Root agent (required by ADK runner)
root_agent = research_pipeline_agent


# ---------------------------------------------------------------------------
# Session Service & Runner
# ---------------------------------------------------------------------------

session_service = InMemorySessionService()

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
)


# ---------------------------------------------------------------------------
# Helper utilities
# ---------------------------------------------------------------------------

def print_section(title: str, content: str, emoji: str = "📄") -> None:
    """Pretty-print a pipeline stage output."""
    width = 60
    print(f"\n{emoji}  {title}")
    print("─" * width)
    print(content.strip())
    print("─" * width)


# ---------------------------------------------------------------------------
# Core pipeline runner
# ---------------------------------------------------------------------------

async def run_research_pipeline(topic: str) -> dict:
    """
    Run the full research → analysis → report pipeline for a given topic.

    Args:
        topic: The subject to research (e.g. "quantum computing").

    Returns:
        A dict with keys: raw_research, analysis, final_report.
    """
    # Fresh session for each run
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

    print("\n" + "═" * 60)
    print(f"🔬  RESEARCH REPORT PIPELINE")
    print(f"📌  Topic: {topic}")
    print("═" * 60)

    user_message = types.Content(
        role="user",
        parts=[types.Part(text=f"Research topic: {topic}")],
    )

    try:
        # Execute the sequential pipeline
        # runner.run is synchronous but internally manages async sub-agents
        events = list(
            runner.run(
                user_id=USER_ID,
                session_id=SESSION_ID,
                new_message=user_message,
            )
        )

        # Read all stage outputs from session state
        session = await session_service.get_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
        )
        state = session.state if session else {}

        raw_research = state.get("raw_research", "(not found)")
        analysis     = state.get("analysis",     "(not found)")
        final_report = state.get("final_report", "(not found)")

        # ── Display each stage ────────────────────────────────────────────
        print_section("STAGE 1 — Raw Research Notes", raw_research, "📚")
        print_section("STAGE 2 — Analysis & Insights", analysis,    "🔍")
        print_section("STAGE 3 — Final Report (Markdown)", final_report, "✅")

        return {
            "raw_research": raw_research,
            "analysis":     analysis,
            "final_report": final_report,
        }

    except Exception as exc:
        print(f"\n❌  Pipeline error: {exc}")
        raise


# ---------------------------------------------------------------------------
# Optional: save the final report to a .md file
# ---------------------------------------------------------------------------

async def run_and_save(topic: str, output_path: str = "report.md") -> None:
    """Run the pipeline and save the final Markdown report to disk."""
    results = await run_research_pipeline(topic)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(results["final_report"])

    print(f"\n💾  Final report saved to: {output_path}")


# ---------------------------------------------------------------------------
# Main entry point
# ---------------------------------------------------------------------------

async def main() -> None:
    """Run the research pipeline on a sample topic."""

    # ── Change this topic to anything you like ────────────────────────────
    topic = "The impact of large language models on software development"
    # ─────────────────────────────────────────────────────────────────────

    await run_research_pipeline(topic)

    # Uncomment below to also save the report to a file:
    # await run_and_save(topic, output_path="llm_software_dev_report.md")


if __name__ == "__main__":
    # asyncio.run(main())
    await main()



════════════════════════════════════════════════════════════
🔬  RESEARCH REPORT PIPELINE
📌  Topic: The impact of large language models on software development
════════════════════════════════════════════════════════════

📚  STAGE 1 — Raw Research Notes
────────────────────────────────────────────────────────────
### Overview / Background
Large Language Models (LLMs) are deep learning models trained on vast amounts of text data, enabling them to understand, generate, and process human language. Their application in software development involves leveraging these capabilities to automate, assist, and enhance various stages of the software development lifecycle (SDLC), from requirements gathering and design to coding, testing, deployment, and maintenance. This integration aims to improve productivity, reduce errors, accelerate development cycles, and potentially lower costs.

### Key Facts or Statistics
*   **Significant Productivity Gains:** Studies and industry reports, such as those by